In [1]:
%tensorflow_version 1.14.0
from google.colab import drive

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/sunil/concepts_loss/concept_learning/

/content/gdrive/MyDrive/sunil/concepts_loss/concept_learning


In [4]:

# !git diff --name-only

In [5]:
# !git checkout .
# !git pull

In [6]:
import tensorflow as tf
from clearn.experiments.experiment import initialize_model_train_and_get_features, MODEL_TYPE_VAE_SEMI_SUPERVISED_MNIST
from clearn.config import ExperimentConfig
from clearn.utils.data_loader import load_images



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [7]:
from clearn.experiments.experiment import Experiment, initialize_model_train_and_get_features,  MODEL_TYPE_VAE_SEMI_SUPERVISED_MNIST
from copy import copy


In [8]:
create_split = True
experiment_name = "find_architecture_unsupervised_vae_seg_mnist"
num_epochs = 50
run_id = 2
z_dim = 16
train_val_data_iterator = None
root_path="/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp"


In [11]:
!ls   /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_unsupervised_vae_seg_mnist

Exp_128_128_128_256_512_16_2	 Exp_32_64_128_256_512_16_2
Exp_128_256_256_512_16_2	 Exp_32_64_128_512_16_2
Exp_128_512_512_512_512_16_2	 Exp_32_64_128_512_512_512_16_2
Exp_256_256_512_512_512_16_2	 Exp_32_64_256_256_512_16_2
Exp_256_512_512_512_512_16_2	 Exp_32_64_64_128_128_256_16_2
Exp_32_128_128_256_512_16_2	 Exp_32_64_64_128_256_256_16_2
Exp_32_128_256_256_512_16_2	 Exp_32_64_64_256_256_512_16_2
Exp_32_32_128_256_512_16_2	 Exp_32_64_64_256_512_16_2
Exp_32_32_128_512_16_2		 Exp_32_64_64_64_128_256_16_2
Exp_32_32_128_512_512_512_16_2	 Exp_512_512_512_512_512_16_2
Exp_32_32_32_256_256_512_16_2	 Exp_64_128_128_256_512_16_2
Exp_32_32_32_256_512_16_2	 Exp_64_128_256_256_512_16_2
Exp_32_32_32_512_16_2		 Exp_64_128_256_512_16_2
Exp_32_32_64_256_256_512_16_2	 Exp_64_512_512_512_512_16_2
Exp_32_32_64_256_512_16_2	 Exp_64_64_128_256_512_16_2
Exp_32_32_64_512_16_2		 Exp_64_64_128_512_16_2
Exp_32_32_64_64_128_256_16_2	 Exp_64_64_256_256_512_16_2
Exp_32_512_512_512_512_16_2	 Exp_64_64_256_512_16_

In [ ]:

num_units_list_5layer_512_512 = [[512, 512, 512, 512, 512],
                             [512, 512, 512, 512, 256],
                             [512, 512, 512, 512, 128],
                             [512, 512, 512, 512, 64],
                             [512, 512, 512, 512, 32],
                             [512, 512, 512, 256, 256],
                             [512, 512, 512, 256, 128],
                             [512, 512, 512, 256, 64],
                             [512, 512, 512, 256, 32],
                             [512, 512, 512, 128, 64],
                             [512, 512, 512, 128, 32],
                             [512, 256, 256, 64, 64],
                             [512, 256, 256, 64, 32],
                             [512, 256, 256, 32, 32],

                  ]

num_units_list_5layer_512_256_0 = [ [512, 256, 128, 128, 128],
                             [512, 256, 128, 128, 64],
                             [512, 256, 128, 128, 32],
                             [512, 256, 128, 64, 64],
                             [512, 256, 128, 64, 32],
                             [512, 256, 128, 32, 32],
                             [512, 256, 64, 64, 64],
                             [512, 256, 64, 64, 32],
                             [512, 256, 64, 32, 32],
                             [512, 256, 32, 32, 32],
                             ]


num_units_list_5layer_512_256_256 = [ [512, 256, 256, 256, 256],
                             [512, 256, 256, 256, 128],
                             [512, 256, 256, 256, 64],
                             [512, 256, 256, 256, 32],
                             [512, 256, 256, 128, 128],
                             [512, 256, 256, 128, 32],
                             [512, 256, 256, 64, 32],
                             [512, 256, 256, 32, 32],
                             [512, 256, 128, 64, 64],
                             ]


num_units_list_5layer_512_256_1 = [[512, 256, 256, 128, 64],
                                   [512, 256, 256, 128, 32],
                                   [512, 256, 256, 64, 64],
                                   [512, 256, 256, 64, 32]
                                   ]
import copy 

num_units_list_5layer_512_256 = copy.deepcopy(num_units_list_5layer_512_256_0)
num_units_list_5layer_512_256.extend(num_units_list_5layer_512_256_1)



stride_5_layer = [2, 1, 2, 1,2,  1]

num_units_list_4layer_512 = [[512, 256, 256, 128],
                  [512, 256, 128, 64],
                  [512, 256, 64, 64],
                  [512, 128, 64, 64],
                  [512, 128, 64, 32],
                  [512, 128, 32, 32],
                  [512, 64, 32, 32],
                  [512, 32, 32, 32]
                  ]
stride_4_layer = [2, 2, 2, 1, 1]


num_units_list_4layer_256 = [
                  [256, 256, 128, 64],
                  [256, 256, 128, 32],
                  [256, 256, 64, 64],
                  [256, 256, 32, 32],
                  [256, 128, 128, 64],
                  [256, 128, 128, 32],
                  [256, 128, 64, 64],
                  [256, 128, 64, 32],
                  [256, 128, 32, 32],
                  [256, 64, 64, 64],
                  [256, 64, 64, 32],
                  [256, 64, 32, 32],
                  [256, 32, 32, 32]
                  
]
num_units_list_5layer_512_256_0 = [[512, 256, 256, 128, 64],
                                 [512, 256, 256, 128, 32],
                                 [512, 256, 256, 64, 64],
                                 [512, 256, 256, 64, 32]
                                 ]
num_units_list_5layer_512_256_1 = [ [512, 256, 128, 128, 128],
                             [512, 256, 128, 128, 64],
                             [512, 256, 128, 128, 32],
                             [512, 256, 128, 64, 64],
                             [512, 256, 128, 64, 32],
                             [512, 256, 128, 32, 32],
                             [512, 256, 64, 64, 64],
                             [512, 256, 64, 64, 32],
                             [512, 256, 64, 32, 32],
                             [512, 256, 32, 32, 32],
                             ] 
import copy
num_units_list_5layer_512_256 = copy.deepcopy(num_units_list_5layer_512_256_0)
num_units_list_5layer_512_256.extend(num_units_list_5layer_512_256_1)


In [ ]:
len(num_units_list_5layer_512_256)
num_units_list_5layer_512_256[10]


[512, 256, 64, 64, 64]

In [ ]:
exp_config = ExperimentConfig(root_path=root_path,
                              num_decoder_layer=4,
                              num_cluster_config=None,
                              supervise_weight=0,
                              name=experiment_name,
                              z_dim=z_dim,
                              batch_size=512,
                              beta=5,
                              reconstruction_weight=1,
                              num_units=num_units_list_5layer_512_256[0],
                              save_reconstructed_images=True,
                              split_name="split_70_30",
                              dataset_name="mnist_concepts",
                              eval_interval_in_epochs=4,
                              model_save_interval=10,
                              num_val_samples=-1,
                                                                                            run_evaluation_during_training=True,
                                                                                            return_latent_vector=False,
                              write_predictions=True,
                                                                                           fully_convolutional = True,
                                                                                           num_concepts=28,
                                                                                           strides=stride_4_layer,
                                                                                           activation_output_layer = "SIGMOID",
                                                                                           num_dense_layers=0,
                                                                                           uncorrelated_features=False,
                              concept_id=20
                                                                                            )
exp_config.check_and_create_directories(run_id)


dataset_name mnist_concepts
/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets/ split_70_30 manually_generated_concepts.json
Reading concepts map from /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets//mnist_concepts/split_70_30/manually_generated_concepts.json
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


True

In [ ]:
exp_config.ANALYSIS_PATH

'/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_unsupervised_vae_seg_mnist/Exp_64_128_256_256_512_16_2/analysis/'

In [ ]:
from clearn.analysis.cluster_utils import get_manual_annotation_col_name
import pandas as pd

In [ ]:
exp_config.dao.num_classes

28

In [ ]:
exp_config.dao.num_concepts_label_generated

18

In [ ]:
from clearn.utils.utils import show_all_variables, get_padding_info

def get_num_concepts_per_image(exp_config, dao):

    _,  _, image_sizes = get_padding_info(exp_config, dao.image_shape)
    latent_image_dim = image_sizes[len(exp_config.num_units)]
    concepts_stride = 1

    if latent_image_dim[0] % concepts_stride == 0:
        h = latent_image_dim[0] //concepts_stride
    else:
        h = (latent_image_dim[0] // concepts_stride) + 1
    if latent_image_dim[1] % concepts_stride == 0:
        w = latent_image_dim[1] // concepts_stride
    else:
        w = (latent_image_dim[1] // concepts_stride) + 1
    return h, w


In [ ]:
from clearn.dao.idao import IDao
from clearn.utils.data_loader import TrainValDataIterator, DataIterator
import os
def get_train_val_iterator(create_split: bool,
                           dao: IDao,
                           exp_config: ExperimentConfig,
                           num_epochs_completed: int,
                           split_name: str):
    split_filename = exp_config.DATASET_PATH + split_name + ".json"
    manual_annotation_file_name = f"manual_annotation.csv"

    manual_annotation_file = os.path.join(exp_config.ANALYSIS_PATH,
                                          manual_annotation_file_name
                                          )
    if exp_config.fully_convolutional:
        num_concepts_per_row, num_concepts_per_col = get_num_concepts_per_image(exp_config, dao)
    else:
        num_concepts_per_row, num_concepts_per_col = 1, 1
    if os.path.isfile(split_filename):
        print("File exists", split_filename)
        if manual_annotation_file is not None:
            train_val_data_iterator = TrainValDataIterator.from_existing_split(dao=dao,
                                                                               split_name=exp_config.split_name,
                                                                               split_location=exp_config.DATASET_PATH,
                                                                               batch_size=exp_config.BATCH_SIZE,
                                                                               manual_labels_config=exp_config.manual_labels_config,
                                                                               manual_annotation_file=manual_annotation_file,
                                                                               budget=exp_config.budget,
                                                                               num_concepts_per_image_row=num_concepts_per_row,
                                                                               num_concepts_per_image_col=num_concepts_per_col
                                                                               )
    elif create_split:
        train_val_data_iterator = TrainValDataIterator(dataset_path=exp_config.DATASET_ROOT_PATH,
                                                       dao=dao,
                                                       shuffle=True,
                                                       stratified=True,
                                                       validation_samples=exp_config.num_val_samples,
                                                       split_names=["train", "validation"],
                                                       split_location=exp_config.DATASET_PATH,
                                                       batch_size=exp_config.BATCH_SIZE,
                                                       manual_labels_config=exp_config.manual_labels_config,
                                                       manual_annotation_file=manual_annotation_file,
                                                       seed=exp_config.seed,
                                                       budget=exp_config.budget,
                                                       num_concepts_per_image_row=num_concepts_per_row,
                                                       num_concepts_per_image_col=num_concepts_per_col
                                                       )
    else:
        raise Exception(f"File does not exists {split_filename}")
    return train_val_data_iterator


In [ ]:
exp_config.ANALYSIS_PATH

'/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_unsupervised_vae_seg_mnist/Exp_64_128_256_256_512_16_2/analysis/'

In [ ]:
# train_val_data_iterator = get_train_val_iterator(create_split=True,
#                                                 dao= exp_config.dao,
#                                                 exp_config= exp_config,
#                                                 num_epochs_completed=0,
#                                                 split_name=exp_config.split_name
#                                                  )


In [ ]:
df = pd.read_csv(exp_config.root_path+ "/datasets/" + f"mnistconcept_{exp_config.concept_id}.csv")

In [ ]:
df.shape

(104878, 785)

In [ ]:
# import pandas as pd
# import os
# manual_annotation_file = os.path.join(exp_config.ANALYSIS_PATH, "manual_annotation.csv")
# # epoch_col_confidence, epoch_col_label = get_manual_annotation_col_name(0.0)

# manual_annotation_df = pd.DataFrame(df["label"].values,
#                                     columns=["manual_annotation"] )
# manual_annotation_df["manual_annotation_confidence"] = 1
# manual_annotation_df.to_csv(manual_annotation_file, index=False)


In [ ]:
# manual_annotation_df.shape

In [ ]:
# import numpy as np
# np.max(manual_annotation_df["manual_annotation"])

In [ ]:
exp_config.dao.num_classes

28

In [ ]:
!ls /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_unsupervised_vae_seg_mnist

Exp_128_128_128_256_512_16_2	 Exp_32_64_128_256_512_16_2
Exp_128_256_256_512_16_2	 Exp_32_64_128_512_16_2
Exp_128_512_512_512_512_16_2	 Exp_32_64_256_256_512_16_2
Exp_256_512_512_512_512_16_2	 Exp_512_512_512_512_512_16_2
Exp_32_128_128_256_512_16_2	 Exp_64_128_128_256_512_16_2
Exp_32_128_256_256_512_16_2	 Exp_64_128_256_256_512_16_2
Exp_32_128_512_512_512_512_16_2  Exp_64_128_256_512_16_2
Exp_32_32_128_256_512_16_2	 Exp_64_512_512_512_512_16_2
Exp_32_32_128_512_16_2		 Exp_64_64_128_256_512_16_2
Exp_32_32_32_512_16_2		 Exp_64_64_128_512_16_2
Exp_32_32_64_512_16_2		 Exp_64_64_256_256_512_16_2
Exp_32_512_512_512_512_512_16_2  Exp_64_64_256_512_16_2


In [ ]:
# !rm -rf "/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_unsupervised_vae_seg_mnist/Exp_128_256_256_512_16_2"


In [ ]:

# train_images, train_labels, _ = load_images(exp_config,
#                                           train_val_data_iterator,
#                                           "train"
#                                           )
#train_labels= np.argwhere(train_labels == 1)[:,1]

In [ ]:
train_val_data_iterator.reset_counter("train")

In [ ]:
train_val_data_iterator.get_num_samples("train")

73414

In [ ]:
df.shape

(104878, 785)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,252.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [ ]:
import numpy as np
print(df.shape)
np.max(df.values[:, 0:784])

(104878, 785)


255.0

In [ ]:
np.max(train_val_data_iterator.train_x)

1.0

In [ ]:
#!git diff --name-only

In [ ]:
train_val_data_iterator.manual_annotation.shape

(104878, 29)

# New Section

In [ ]:
tf.reset_default_graph()
for num_units in num_units_list_5layer_512_256[10:]:
      train_val_data_iterator, exp_config, model = initialize_model_train_and_get_features(root_path=root_path,
                                                                                           experiment_name=experiment_name,
                                                                                           z_dim=z_dim,
                                                                                           batch_size=512,
                                                                                           run_id=run_id,
                                                                                           beta=5,
                                                                                           create_split=create_split,
                                                                                           num_epochs=num_epochs,
                                                                                           supervise_weight=0,
                                                                                           reconstruction_weight=1,
                                                                                           model_type=MODEL_TYPE_VAE_SEMI_SUPERVISED_MNIST,
                                                                                           num_units=num_units,
                                                                                           save_reconstructed_images=True,
                                                                                           split_name="split_70_30",
                                                                                           dataset_name="mnist_concepts",
                                                                                           train_val_data_iterator=train_val_data_iterator,
                                                                                            eval_interval_in_epochs=4,
                                                                                            model_save_interval=10,
                                                                                            num_val_samples=-1,
                                                                                            run_evaluation_during_training=True,
                                                                                            return_latent_vector=False,
                                                                                            write_predictions=True,
                                                                                           fully_convolutional = True,
                                                                                           num_concepts=28,
                                                                                           strides=stride_5_layer,
                                                                                           activation_output_layer = "SIGMOID",
                                                                                           num_dense_layers=0,
                                                                                           supervise_weight_concepts=0,
                                                                                           uncorrelated_features=False,
                                                                                           concept_id=20
                                                                                            )
      tf.reset_default_graph()



Streaming output truncated to the last 5000 lines.
Epoch: 10/59, Loss:133.3179931640625 Nll_loss : 98.21182250976562 KLD:7.021235942840576  Supervised loss:3.6344292163848877 Supervised loss concepts:0.0
Epoch: 10/60, Loss:132.2471923828125 Nll_loss : 96.99272155761719 KLD:7.050892353057861  Supervised loss:3.6781258583068848 Supervised loss concepts:0.0
Epoch: 10/61, Loss:135.18093872070312 Nll_loss : 100.64157104492188 KLD:6.907871723175049  Supervised loss:3.650193452835083 Supervised loss concepts:0.0
Epoch: 10/62, Loss:134.5299072265625 Nll_loss : 99.02989196777344 KLD:7.100003242492676  Supervised loss:3.649578094482422 Supervised loss concepts:0.0
Epoch: 10/63, Loss:131.0531005859375 Nll_loss : 95.93763732910156 KLD:7.0230937004089355  Supervised loss:3.643958806991577 Supervised loss concepts:0.0
Epoch: 10/64, Loss:134.133056640625 Nll_loss : 98.2209701538086 KLD:7.182417869567871  Supervised loss:3.6627931594848633 Supervised loss concepts:0.0
Epoch: 10/65, Loss:134.2409667968

In [ ]:
df.head(100)

In [ ]:
drive.flush_and_unmount()